In [1]:
import os

os.chdir("../../")

In [2]:
import mlflow
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec

from src.io.path_definition import get_project_dir


input_schema = Schema([ColSpec("string", "input")])
output_schema = Schema([ColSpec("string", "input"),
                        ColSpec("string", "output")])
signature = ModelSignature(inputs=input_schema, outputs=output_schema)


model_path = os.path.join(get_project_dir(), 'src', 'agent', "agent_mlflow_experiment.py")

mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

"""
The registry is a separate feature: you must either (a) log with a registered_model_name argument, 
or (b) promote a logged model to the registry manually in the UI/CLI.
"""

with mlflow.start_run(run_name="2025-08-27-test-2") as run:

    run_id = run.info.run_id
    
    # Save run_id somewhere accessible (env variable, file, argument, etc.)
    os.environ["MLFLOW_RUN_ID"] = run_id
    
    mlflow.log_artifact(model_path, artifact_path="source_code")
    model_info = mlflow.pyfunc.log_model(
        python_model=model_path,  # Define the model as the path to the Python file
        name="my_model",
        input_example={"input": "What is the capital of France?"},
        signature=signature,
        registered_model_name="zeroshot_react_agent"
    )

C:\Users\omega\miniconda3\envs\agent\lib\site-packages\mlflow\pyfunc\utils\data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
2025/08/27 16:37:38 INFO mlflow.pyfunc: Validating input example against model signature
C:\Users\omega\miniconda3\envs\agent\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\omega\miniconda3\envs\agent\lib\site-packages\mlflow\pyfunc\utils\data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.ht



> Entering new AgentExecutor chain...
Action: Search Engine

Action Input: "capital of France"

## [Paris, France](https://www.google.com/maps/search/Paris%2C+France)


Paris is the capital and largest city of France, located in the north-central part of the country along the Seine River. As of January 2025, the city has a population of approximately 2,048,472 residents. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Paris?utm_source=openai)) Paris is renowned for its rich history, cultural landmarks, and significant influence in art, fashion, and gastronomy.

In 2024, Paris hosted the Summer Olympic Games from July 26 to August 11, marking the third time the city has held the event, following the 1900 and 1924 Games. ([time.com](https://time.com/7003861/paris-olympics-2024-host-city-surprising-facts/?utm_source=openai)) The Olympics brought a wave of enthusiasm to the city, transforming its streets and engaging both residents and visitors. ([lemonde.fr](https://www.lemonde.fr/en/

2025/08/27 16:40:07 INFO mlflow.models.model: Found the following environment variables used during model inference: [DEEPSEEK_API_KEY, GOOGLE_API_KEY, HUGGINGFACE_API_KEY, ... ]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


I now know the final answer.

Final Answer: The capital of France is Paris.

> Finished chain.


Registered model 'zeroshot_react_agent' already exists. Creating a new version of this model...
2025/08/27 16:40:09 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: zeroshot_react_agent, version 7
Created version '7' of model 'zeroshot_react_agent'.


🏃 View run 2025-08-27-test-2 at: http://127.0.0.1:8080/#/experiments/0/runs/50465efe2130480bb5496b875fddb89f
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0


In [ ]:
mlflow.pyfunc.log_model?

In [ ]:
model_info.model_uri

In [ ]:
my_model = mlflow.pyfunc.load_model(model_info.model_uri)

In [ ]:
my_model.predict({"input": "what is the hometown of the mens 2023 Australia open winner?"})

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

from src.initialization import credential_init

credential_init()

llm_gpt_4o_mini = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                             model_name="gpt-4o-mini", temperature=0)



In [ ]:
from openai import OpenAI
from langchain_core.runnables import chain
from langchain.tools import BaseTool


@chain
def gpt_web_search_tool(text):

    client = OpenAI()

    response = client.chat.completions.create(
        model="gpt-4o-search-preview",
        web_search_options={"search_context_size": "medium"},
        messages=[{"role": "user",
                   "content": text}]
    )

    return response.choices[0].message.content


class SearchTool(BaseTool):

    name: str = "Search Engine"
    description: str = "Use this tool to find the knowledge you need."

    def _run(self, query: str):
        
        response = gpt_web_search_tool.invoke(query)
        
        return response

    async def _arun(self, query: str):
        response = await gpt_web_search_tool.ainvoke(query)
        
        return response

In [ ]:
import mlflow
import mlflow.pyfunc
from langchain.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent

from src.agent.react_zero_shot import prompt_template as zero_shot_prompt_template

"""
You cannot directly log an AgentExecutor using mlflow.langchain.log_model. 
That API is designed for basic LangChain components, not agents or arbitrary Runnable graphs.
"""

class AgentModel(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        # rebuild your agent here
        prompt = PromptTemplate(template=zero_shot_prompt_template)

        tools=[SearchTool()]
        
        zero_shot_agent = create_react_agent(
            llm=llm_gpt_4o_mini,
            tools=tools,
            prompt=prompt,
        )
        self.agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True,
                                       handle_parsing_errors=True)

    def predict(self, context, model_input):
        return self.agent_executor.invoke({"input": model_input["input"]})

with mlflow.start_run():
    mlflow.pyfunc.log_model(
        artifact_path="zeroshot_react_agent",
        python_model=AgentModel(),
        input_example={"input": "What is the capital of France?"},
    )

## Define the State

Let's now start by defining the state the track for this agent.

First, we will need to track the current plan. Let's represent that as a list of strings.

Next, we should track previously executed steps. Let's represent that as a list of tuples (these tuples will contain the step and then the result)

Finally, we need to have some state to represent the final response as well as the original input.

In [ ]:
import operator
from typing import Annotated, List, Tuple
from typing_extensions import TypedDict


class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: List[Tuple]
    response: str

In [ ]:
from typing import List, Literal, Union
from pydantic import BaseModel, Field

from langchain_core.prompts import ChatPromptTemplate


class Plan(BaseModel):
    """Plan to follow in future"""
    type: Literal["plan"] = "plan"  # discriminator
    steps: List[str] = Field(
        description="different steps to follow, should be in sorted order"
    )



planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.""",
        ),
        ("placeholder", "{messages}"),
    ]
)
planner = planner_prompt | llm_gpt_4o_mini.with_structured_output(Plan)

In [ ]:
planner.invoke(
    {
        "messages": [
            ("user", "what is the hometown of the current Australia open winner?")
        ]
    }
)

In [ ]:
from typing import Union, Optional
from textwrap import dedent

from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate


class Response(BaseModel):
    """Response to user."""
    type: Literal["response"] = "response"  # discriminator
    response: str = Field(description="The answer to the objective.")


class Act(BaseModel):
    """Action to perform."""

    response: Optional[str] = Field(
        None, description="Direct response to the user."
    )
    steps: Optional[List[str]] = Field(
        None,
        description="Plan steps to follow, in sorted order."
    )


def build_standard_chat_prompt_template(kwargs):

    system_content = kwargs['system']
    human_content = kwargs['human']
    
    system_prompt = PromptTemplate(**system_content)
    system_message = SystemMessagePromptTemplate(prompt=system_prompt)
    
    human_prompt = PromptTemplate(**human_content)
    human_message = HumanMessagePromptTemplate(prompt=human_prompt)
    
    chat_prompt = ChatPromptTemplate.from_messages([system_message,
                                                     human_message
                                                   ])

    return chat_prompt


system_template = dedent("""
For the given objective, come up with a simple step by step plan.
This plan should involve individual tasks, that if executed correctly will yield the correct answer. 
Do not add any superfluous steps. The result of the final step should be the final answer. 
Make sure that each step has all the information needed - do not skip steps.

You will be provided with

1. the question you have to response
2. the current object
3. the plan
4. what you have done, which is a list of (`previous plan`, `previous action`)

If you come up with the answer to the objective, please do not confirm the result and 
`Response` to user. If plan is an empty list, you `Response` to the user based on what have done. 
Otherwise, update the plan accodingly. Only add steps to the plan that still NEED to 
be done. Do not repeat the previous plan.
""")

human_template = dedent("""
Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the follow steps:
{past_steps}
""")

input_ = {"system": {"template": system_template},
          "human": {"template": human_template}}
    
replanner_prompt = build_standard_chat_prompt_template(input_)

replanner = replanner_prompt | llm_gpt_4o_mini.with_structured_output(Act)

In [ ]:
system_template = dedent("""
For the given objective, generate the answer based on the steps.
""")

human_template = dedent("""
Your objective was this:
{input}

You have currently done the follow steps:
{past_steps}
""")

input_ = {"system": {"template": system_template},
          "human": {"template": human_template}}
    
replanner_prompt = build_standard_chat_prompt_template(input_)

replanner = replanner_prompt | llm_gpt_4o_mini.with_structured_output(Response)

In [ ]:
from typing import Literal
from langgraph.graph import END


async def execute_step(state: PlanExecute):
    plan = state["plan"]
    plan_str = "\n".join(f"{i + 1}. {step}" for i, step in enumerate(plan))
    task = plan[0]
    task_formatted = dedent(f"""For the following plan:
                                {plan_str}\n\nYou are tasked with executing step {1}, {task}.""")
    agent_response = await agent_executor.ainvoke(
        {"input": task_formatted}
    )
    if 'past_steps' not in state:
        return {"past_steps": [(task, agent_response["output"])]}
    else:
        return {
            "past_steps": state['past_steps'] + [(task, agent_response["output"])]
        }


async def plan_step(state: PlanExecute):
    plan = await planner.ainvoke({"messages": [("user", state["input"])]})
    return {"plan": plan.steps}


async def replan_step(state: PlanExecute):
    output = await replanner.ainvoke(state)
    print("**********************")
    print(output)
    print("**********************")
    if output.response:
        return {"response": output.response}
    elif output.steps:
        return {"plan": output.steps}
    else:
        output = await responser.ainvoke(state)
        return {"response": output.response}
    
def should_end(state: PlanExecute):
    if "response" in state and state["response"]:
        return END
    else:
        return "agent"

In [ ]:
from langgraph.graph import StateGraph, START

workflow = StateGraph(PlanExecute)

# Add the plan node
workflow.add_node("planner", plan_step)

# Add the execution step
workflow.add_node("agent", execute_step)

# Add a replan node
workflow.add_node("replan", replan_step)

workflow.add_edge(START, "planner")

# From plan we go to agent
workflow.add_edge("planner", "agent")

# From agent, we replan
workflow.add_edge("agent", "replan")

workflow.add_conditional_edges(
    "replan",
    # Next, we pass in the function that will determine which node is called next.
    should_end,
    ["agent", END],
)

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [ ]:
from IPython.display import Image, display

display(Image(app.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
config = {"recursion_limit": 50}
inputs = {"input": "what is the hometown of the mens 2023 Australia open winner?"}


In [ ]:
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)

In [ ]:
config = {"recursion_limit": 50}
inputs = {"input": "Help me prepare potential interview questions for a staff assuming that topic is about Transfomers?"}

In [ ]:
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)